In [1]:
  
from __future__ import print_function

from keras import backend as K

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def

from keras.optimizers import SGD
from keras.models import model_from_json
#from keras.models import Model

import shutil
import os
import numpy as np
import tensorflow as tf


Using TensorFlow backend.
/home/iago/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iago/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iago/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iago/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [16]:
json_file = open("purchase_classification_20200205.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into model
loaded_model.load_weights("purchase_classification_20200205.h5")

sgd = SGD(lr=0.1)
loaded_model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

print("Loaded model from disk")

from keras import backend as K
from keras.models import load_model
import tensorflow as tf

RuntimeError: Attempted to use a closed Session.

In [ ]:
# prepare model for export

"""
K.set_learning_phase(0)
config = loaded_model.get_config()
weights = loaded_model.get_weights()
model = Model.from_config(config)
model.set_weights(weights)
"""

In [ ]:
model = loaded_model

# export model
if os.path.isdir("./export"):
    shutil.rmtree("./export")
    
export_path = "./export"

builder = saved_model_builder.SavedModelBuilder(export_path)

print(model.input)
print(model.output)

In [18]:
signature = predict_signature_def(inputs={"inputs": model.input},
                                  outputs={"outputs": model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,tags=[tag_constants.SERVING],signature_def_map={'predict': signature})
    builder.save()

RuntimeError: Attempted to use a closed Session.